In [15]:
import os
import openai



from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.0)

/Users/altaf/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [39]:
input_template = """ 
You are an automated translation system, \
which translates the text delimited by triple backticks \
to {language} and expand it as a {style} \
Format the output as JSON with the following keys:
input_text
input_language
output_text
output_language
output_style
text: ```{text}```

{format_instructions}
"""

In [25]:
prompt_template = ChatPromptTemplate.from_template(input_template)
print(prompt_template)
prompt_template.messages[0].prompt

input_variables=['language', 'text', 'style'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'style', 'text'], output_parser=None, partial_variables={}, template=' \nYou are an automated translation system, which translates the text delimited by triple backticks to {language} and expand it as a {style} Format the output as JSON with the following keys:\ninput_text\ninput_language\noutput_text\noutput_language\noutput_style\ntext: ```{text}```\n', template_format='f-string', validate_template=True), additional_kwargs={})]


PromptTemplate(input_variables=['language', 'style', 'text'], output_parser=None, partial_variables={}, template=' \nYou are an automated translation system, which translates the text delimited by triple backticks to {language} and expand it as a {style} Format the output as JSON with the following keys:\ninput_text\ninput_language\noutput_text\noutput_language\noutput_style\ntext: ```{text}```\n', template_format='f-string', validate_template=True)

In [26]:
input_msg = """ 
Today is a beautiful day"""

output_language = """ 
English"""

style_used = """ 
limrick"""

In [27]:
chat_input = prompt_template.format_messages(
                    language=output_language,
                    text=input_msg,
                    style = style_used)

print(chat_input[0])

content=' \nYou are an automated translation system, which translates the text delimited by triple backticks to  \nEnglish and expand it as a  \nlimrick Format the output as JSON with the following keys:\ninput_text\ninput_language\noutput_text\noutput_language\noutput_style\ntext: ``` \nToday is a beautiful day```\n' additional_kwargs={} example=False


In [28]:
response = chat(chat_input)

In [29]:
print(response)

content='{\n  "input_text": "Today is a beautiful day",\n  "input_language": "unknown",\n  "output_text": "There once was a day so fine,\\nWith sunshine and blue skies divine,\\nIt filled me with glee,\\nAnd made me feel free,\\nOh how I wish it could be mine!",\n  "output_language": "en",\n  "output_style": "limerick"\n}' additional_kwargs={} example=False


In [32]:
print(response.content)
print(type(response.content))

{
  "input_text": "Today is a beautiful day",
  "input_language": "unknown",
  "output_text": "There once was a day so fine,\nWith sunshine and blue skies divine,\nIt filled me with glee,\nAnd made me feel free,\nOh how I wish it could be mine!",
  "output_language": "en",
  "output_style": "limerick"
}
<class 'str'>


In [45]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser


input_text_schema = ResponseSchema(name = "input_text",
                                   description= "the text to translate")

input_language_schema = ResponseSchema(name = "input_language",
                                       description= "the langauge of the input text")

output_text_schema = ResponseSchema(name= "output_text",
                                    description="the output text")

output_language_schema = ResponseSchema(name= "output_language",
                                        description="the language of the output")

output_style_schema = ResponseSchema(name = "output_style",
                                     description= "the style used for the output")

response_schemas = [input_text_schema,
                    input_language_schema,
                    output_text_schema,
                    output_language_schema,
                    output_style_schema]

In [46]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [47]:
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"input_text": string  // the text to translate
	"input_language": string  // the langauge of the input text
	"output_text": string  // the output text
	"output_language": string  // the language of the output
	"output_style": string  // the style used for the output
}
```


In [48]:

prompt_template = ChatPromptTemplate.from_template(input_template)

chat_input = prompt_template.format_messages(
                    language=output_language,
                    text=input_msg,
                    style = style_used,
                    format_instructions=format_instructions)

print(chat_input[0].content)


 
You are an automated translation system, which translates the text delimited by triple backticks to  
English and expand it as a  
limrick Format the output as JSON with the following keys:
input_text
input_language
output_text
output_language
output_style
text: ``` 
Today is a beautiful day```

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"input_text": string  // the text to translate
	"input_language": string  // the langauge of the input text
	"output_text": string  // the output text
	"output_language": string  // the language of the output
	"output_style": string  // the style used for the output
}
```



In [49]:
response = chat(chat_input)

print(response.content)

```json
{
	"input_text": "Today is a beautiful day",
	"input_language": "English",
	"output_text": "There once was a day so fine,\nWith sunshine and warmth so divine,\nIt's beauty did shine,\nAnd all felt just fine,\nOh what a day, so sublime!",
	"output_language": "English",
	"output_style": "limerick"
}
```


In [50]:
output_dict = output_parser.parse(response.content)

output_dict

{'input_text': 'Today is a beautiful day',
 'input_language': 'English',
 'output_text': "There once was a day so fine,\nWith sunshine and warmth so divine,\nIt's beauty did shine,\nAnd all felt just fine,\nOh what a day, so sublime!",
 'output_language': 'English',
 'output_style': 'limerick'}